In [51]:
BASE_DIR = '../'*3
DATASET_DIR = BASE_DIR+'code/util/dataset/dataset_dummy.ipynb'
EXTMODEL_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [135]:
%run {DATASET_DIR}
%run {EXTMODEL_DIR}

Using device: cpu
Using PyTorch version: 1.9.0


In [136]:
imagenet = DummyDataset('imagenet', 'select',[3,224,224],1000)

1000


In [137]:
CnnExtModel.set_macro('p24',
    ['serial',
        ['loop', {'repeat':'#repeat'}, ['conv', {'ksize':3, 'chn':'#chn'}]],
        ['max', {'stride':2}]])

CnnExtModel.set_macro('vgg_19',
    ['serial',
        ['custom', {'name':'p24', 'args':{'#repeat':2, '#chn':64}}],
        ['custom', {'name':'p24', 'args':{'#repeat':2, '#chn':128}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':256}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':512}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':512}}],
        ['loop', {'repeat':2}, ['full', {'width':4096}]]])

vgg19 = CnnExtModel('vgg_19', imagenet,
        ['custom', {'name':'vgg_19'}], dump_structure=True)

custom vgg_19
  serial
    custom p24
      serial
        loop
          1: conv, [3, 224, 224]=>[64, 224, 224]pm : 64x3x3x3+64 = 1792
          2: conv, [64, 224, 224]=>[64, 224, 224]pm : 64x64x3x3+64 = 36928
        3: max, [64, 224, 224]=>[64, 112, 112]
[[[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)], [MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)]]]
    custom p24
      serial
        loop
          4: conv, [64, 112, 112]=>[128, 112, 112]pm : 128x64x3x3+128 = 73856
          5: conv, [128, 112, 112]=>[128, 112, 112]pm : 128x128x3x3+128 = 147584
        6: max, [128, 112, 112]=>[128, 56, 56]
[[[Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)], [MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)]]]
    custom p24
      serial
        loop
      

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
vgg19.net

In [61]:
CnnExtModel.set_macro('pn',
    ['serial',
        ['conv', {'ksize':3, 'stride':2, 'chn':'#n', 'actions':'#act'}],
        ['loop', {'repeat':'#cnt1'},
                 ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]]])

CnnExtModel.set_macro('plain_34',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['loop', {'repeat':6}, ['conv', {'ksize':3, 'chn':64, 'actions':'#act'}]],
        ['custom', {'name':'pn', 'args':{'#cnt1':7, '#n':128, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':11, '#n':256, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':5, '#n':512, '#act':'#act'}}],
        ['avg', {'stride':7}]])

plain_34 = CnnExtModel('plain_34', imagenet,
       ['custom', {'name':'plain_34', 'args':{'#act':'LA'}}], dump_structure=True)

custom plain_34
  serial
    1: conv, [3, 224, 224]=>[64, 112, 112]pm : 64x3x7x7+64 = 9472
    2: max, [64, 112, 112]=>[64, 56, 56]
    loop
      3: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
      4: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
      5: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
      6: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
      7: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
      8: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
    custom pn
      serial
        9: conv, [64, 56, 56]=>[128, 28, 28]pm : 128x64x3x3+128 = 73856
        loop
          10: conv, [128, 28, 28]=>[128, 28, 28]pm : 128x128x3x3+128 = 147584
          11: conv, [128, 28, 28]=>[128, 28, 28]pm : 128x128x3x3+128 = 147584
          12: conv, [128, 28, 28]=>[128, 28, 28]pm : 128x128x3x3+128 = 147584
          13: conv, [128, 28, 28]=>[128, 28, 28]pm : 128x128x3x3+128 = 147584
          14: conv, [128, 28, 28]

In [62]:
CnnExtModel.set_macro('rf',
    ['add', {'x':True},
        ['serial', ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}],
                   ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]]])

CnnExtModel.set_macro('rh',
    ['add', {'x':False},
        ['serial', ['conv', {'ksize':3, 'stride':2, 'chn':'#n', 'actions':'#act'}],
                   ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]],
        ['avg', {'stride':2}]])

CnnExtModel.set_macro('rfull',
    ['serial',
        ['loop', {'repeat':'#cnt'},
                 ['custom', {'name':'rf', 'args':{'#n':'#n', '#act':'#act'}}]]])

CnnExtModel.set_macro('rhalf',
    ['serial',
        ['custom', {'name':'rh', 'args':{'#n':'#n', '#act':'#act'}}],
        ['loop', {'repeat':'#cnt1'},
                 ['custom', {'name':'rf', 'args':{'#n':'#n', '#act':'#act'}}]]])

CnnExtModel.set_macro('residual_34',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['custom', {'name':'rfull', 'args':{'#cnt':3, '#n':64, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':3, '#n':128, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':5, '#n':256, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':2, '#n':512, '#act':'#act'}}],
        ['avg', {'stride':7}]])

residual_34 = CnnExtModel('residual_34', imagenet,
    ['custom', {'name':'residual_34', 'args':{'#act':'LA'}}], dump_structure=True)

custom residual_34
  serial
    1: conv, [3, 224, 224]=>[64, 112, 112]pm : 64x3x7x7+64 = 9472
    2: max, [64, 112, 112]=>[64, 56, 56]
    custom rfull
      serial
        loop
          custom rf
            add
              serial
                3: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                4: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
          custom rf
            add
              serial
                5: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                6: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
          custom rf
            add
              serial
                7: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                8: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
    custom rhalf
      serial
        custom rh
          add
            serial
              9: conv, [64, 56, 56]=>[128, 28, 28]pm : 128x64x3x3+128 = 73856
              10: conv, [128, 

In [63]:
CnnExtModel.set_macro('bf',
    ['add', {'x':True},
        ['serial',
            ['conv', {'ksize':1, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':1, 'chn':'#n4', 'actions':'#act'}]]])

CnnExtModel.set_macro('bh',
    ['add', {'x':False},
        ['serial',
            ['conv', {'ksize':1, 'stride':2, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':1, 'chn':'#n4', 'actions':'#act'}]],
        ['avg', {'stride':2}]])

CnnExtModel.set_macro('bfull',
    ['serial',
        ['loop', {'repeat':'#cnt'},
            ['custom', {'name':'bf', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}]]])

CnnExtModel.set_macro('bhalf',
    ['serial',
        ['custom', {'name':'bh', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}],
        ['loop', {'repeat':'#cnt1'},
            ['custom', {'name':'bf', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}]]])

CnnExtModel.set_macro('bottleneck_152',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'ksize':3, 'stride':2}],
        ['custom', {'name':'bfull','args':{'#cnt':3,'#n1':64,'#n4':256,'#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':7,'#n1':128,'#n4':512,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':35,'#n1':256,'#n4':1024,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':2,'#n1':512,'#n4':2048,
            '#act':'#act'}}],
        ['avg', {'stride':7}]])

bottleneck_152 = CnnExtModel('bottleneck_152', imagenet,
        ['custom', {'name':'bottleneck_152', 'args':{'#act':'LAB'}}],
                    dump_structure=True)

custom bottleneck_152
  serial
    1: conv, [3, 224, 224]=>[64, 112, 112]pm : 64x3x7x7+64 = 9472
    2: max, [64, 112, 112]=>[64, 56, 56]
    custom bfull
      serial
        loop
          custom bf
            add
              serial
                3: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x1x1+64 = 4160
                4: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                5: conv, [64, 56, 56]=>[256, 56, 56]pm : 256x64x1x1+256 = 16640
          custom bf
            add
              serial
                6: conv, [256, 56, 56]=>[64, 56, 56]pm : 64x256x1x1+64 = 16448
                7: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                8: conv, [64, 56, 56]=>[256, 56, 56]pm : 256x64x1x1+256 = 16640
          custom bf
            add
              serial
                9: conv, [256, 56, 56]=>[64, 56, 56]pm : 64x256x1x1+64 = 16448
                10: conv, [64, 56, 56]=>[64, 56, 56]pm : 64x64x3x3+64 = 36928
                11: co

                83: conv, [1024, 14, 14]=>[256, 14, 14]pm : 256x1024x1x1+256 = 262400
                84: conv, [256, 14, 14]=>[256, 14, 14]pm : 256x256x3x3+256 = 590080
                85: conv, [256, 14, 14]=>[1024, 14, 14]pm : 1024x256x1x1+1024 = 263168
          custom bf
            add
              serial
                86: conv, [1024, 14, 14]=>[256, 14, 14]pm : 256x1024x1x1+256 = 262400
                87: conv, [256, 14, 14]=>[256, 14, 14]pm : 256x256x3x3+256 = 590080
                88: conv, [256, 14, 14]=>[1024, 14, 14]pm : 1024x256x1x1+1024 = 263168
          custom bf
            add
              serial
                89: conv, [1024, 14, 14]=>[256, 14, 14]pm : 256x1024x1x1+256 = 262400
                90: conv, [256, 14, 14]=>[256, 14, 14]pm : 256x256x3x3+256 = 590080
                91: conv, [256, 14, 14]=>[1024, 14, 14]pm : 1024x256x1x1+1024 = 263168
          custom bf
            add
              serial
                92: conv, [1024, 14, 14]=>[256, 14, 14]pm 

In [67]:
bottleneck_152.net

[P_Block(
   (bf_1): Sequential(
     (dict): ModuleDict(
       (bf_1_1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bf_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (bf_1_3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), padding=same)
     )
   )
 ),
 ReLU(),
 P_Block(
   (bf_3): Sequential(
     (dict): ModuleDict(
       (bf_3_1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (bf_3_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (bf_3_3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), padding=same)
     )
   )
 ),
 ReLU(),
 P_Block(
   (bfull_4): Sequential(
     (dict): ModuleDict(
       (bfull_4_1): P_Block(
         (bf_1): Sequential(
           (dict): ModuleDict(
             (bf_1_1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
             (bf_1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
             (bf_1_3)

In [ ]:
# F_DATA_DIR = BASE_DIR+ 'code/util/dataset/flower_prepro.ipynb'

# %run {F_DATA_DIR}

# fd = FlowersDataset([64,64], [3,64,64])

In [ ]:
CnnExtModel.set_macro('plain_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['loop', {'repeat':4}, ['conv', {'ksize':3, 'chn':16, 'actions':'#act'}]],
        ['custom', {'name':'pn', 'args':{'#cnt1':3, '#n':32, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':3, '#n':64, '#act':'#act'}}],
        ['avg', {'stride':4}]])

plain_flower = CnnExtModel('plain_flower', fd,
        ['custom', {'name':'plain_flower', 'args':{'#act':'LAB'}}],
                    dump_structure=True)
plain_flower.exec_all(epoch_count=10, report=2)

In [ ]:
CnnExtModel.set_macro('residual_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['custom', {'name':'rfull', 'args':{'#cnt':2, '#n':16, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':1, '#n':32, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':1, '#n':64, '#act':'#act'}}],
        ['avg', {'stride':4}]])

residual_flower = CnnExtModel('residual_flower', fd,
      ['custom', {'name':'residual_flower', 'args':{'#act':'LAB'}}],
                  dump_structure=True)
residual_flower.exec_all(epoch_count=10, report=2)

In [ ]:
CnnExtModel.set_macro('bottleneck_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'ksize':3, 'stride':2}],
        ['custom', {'name':'bfull', 'args':{'#cnt':1,'#n1':16,'#n4': 64,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf', 'args':{'#cnt1':2,'#n1':32,'#n4':128
            '#act':'#act'}}],
        ['custom', {'name':'bhalf', 'args':{'#cnt1':1,'#n1':64,'#n4':256,
            '#act':'#act'}}],
        ['avg', {'stride':4}]])

bottleneck_flower = CnnExtModel('bottleneck_flower', fd,
    ['custom', {'name':'bottleneck_flower', 'args':{'#act':'LAB'}}],
        dump_structure=True)
bottleneck_flower.exec_all(epoch_count=10, report=2)

In [ ]:
%run {DATASET_DIR}
%run {EXTMODEL_DIR}

imagenet = DummyDataset('imagenet', 'select',[299,299,3],1000)

CnnExtModel.set_macro('p24',
    ['serial',
        ['loop', {'repeat':'#repeat'}, ['conv', {'ksize':3, 'chn':'#chn'}]],
        ['max', {'stride':2}]])

CnnExtModel.set_macro('vgg_19',
    ['serial',
        ['custom', {'name':'p24', 'args':{'#repeat':2, '#chn':64}}],
        ['custom', {'name':'p24', 'args':{'#repeat':2, '#chn':128}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':256}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':512}}],
        ['custom', {'name':'p24', 'args':{'#repeat':4, '#chn':512}}],
        ['loop', {'repeat':2}, ['full', {'width':4096}]]])

vgg19 = CnnExtModel('vgg_19', imagenet,
        ['custom', {'name':'vgg_19'}], dump_structure=True)

CnnExtModel.set_macro('pn',
    ['serial',
        ['conv', {'ksize':3, 'stride':2, 'chn':'#n', 'actions':'#act'}],
        ['loop', {'repeat':'#cnt1'},
                 ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]]])

CnnExtModel.set_macro('plain_34',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['loop', {'repeat':6}, ['conv', {'ksize':3, 'chn':64, 'actions':'#act'}]],
        ['custom', {'name':'pn', 'args':{'#cnt1':7, '#n':128, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':11, '#n':256, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':5, '#n':512, '#act':'#act'}}],
        ['avg', {'stride':7}]])

plain_34 = CnnExtModel('plain_34', imagenet,
       ['custom', {'name':'plain_34', 'args':{'#act':'LA'}}], dump_structure=True)

CnnExtModel.set_macro('rf',
    ['add', {'x':True},
        ['serial', ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}],
                   ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]]])

CnnExtModel.set_macro('rh',
    ['add', {'x':False},
        ['serial', ['conv', {'ksize':3, 'stride':2, 'chn':'#n', 'actions':'#act'}],
                   ['conv', {'ksize':3, 'chn':'#n', 'actions':'#act'}]],
        ['avg', {'stride':2}]])

CnnExtModel.set_macro('rfull',
    ['serial',
        ['loop', {'repeat':'#cnt'},
                 ['custom', {'name':'rf', 'args':{'#n':'#n', '#act':'#act'}}]]])

CnnExtModel.set_macro('rhalf',
    ['serial',
        ['custom', {'name':'rh', 'args':{'#n':'#n', '#act':'#act'}}],
        ['loop', {'repeat':'#cnt1'},
                 ['custom', {'name':'rf', 'args':{'#n':'#n', '#act':'#act'}}]]])

CnnExtModel.set_macro('residual_34',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['custom', {'name':'rfull', 'args':{'#cnt':3, '#n':64, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':3, '#n':128, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':5, '#n':256, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':2, '#n':512, '#act':'#act'}}],
        ['avg', {'stride':7}]])

residual_34 = CnnExtModel('residual_34', imagenet,
    ['custom', {'name':'residual_34', 'args':{'#act':'LA'}}], dump_structure=True)

CnnExtModel.set_macro('bf',
    ['add', {'x':True},
        ['serial',
            ['conv', {'ksize':1, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':1, 'chn':'#n4', 'actions':'#act'}]]])

CnnExtModel.set_macro('bh',
    ['add', {'x':False},
        ['serial',
            ['conv', {'ksize':1, 'stride':2, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':'#n1', 'actions':'#act'}],
            ['conv', {'ksize':1, 'chn':'#n4', 'actions':'#act'}]],
        ['avg', {'stride':2}]])

CnnExtModel.set_macro('bfull',
    ['serial',
        ['loop', {'repeat':'#cnt'},
            ['custom', {'name':'bf', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}]]])

CnnExtModel.set_macro('bhalf',
    ['serial',
        ['custom', {'name':'bh', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}],
        ['loop', {'repeat':'#cnt1'},
            ['custom', {'name':'bf', 'args':{'#n1':'#n1', '#n4':'#n4',
                '#act':'#act'}}]]])

CnnExtModel.set_macro('bottleneck_152',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':64, 'actions':'#act'}],
        ['max', {'ksize':3, 'stride':2}],
        ['custom', {'name':'bfull','args':{'#cnt':3,'#n1':64,'#n4':256,'#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':7,'#n1':128,'#n4':512,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':35,'#n1':256,'#n4':1024,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf','args':{'#cnt1':2,'#n1':512,'#n4':2048,
            '#act':'#act'}}],
        ['avg', {'stride':7}]])

bottleneck_152 = CnnExtModel('bottleneck_152', imagenet,
        ['custom', {'name':'bottleneck_152', 'args':{'#act':'LAB'}}],
                    dump_structure=True)

%run ../chap05/dataset_flowers.ipynb

fd = FlowersDataset([64,64], [64,64,3])

CnnExtModel.set_macro('plain_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['loop', {'repeat':4}, ['conv', {'ksize':3, 'chn':16, 'actions':'#act'}]],
        ['custom', {'name':'pn', 'args':{'#cnt1':3, '#n':32, '#act':'#act'}}],
        ['custom', {'name':'pn', 'args':{'#cnt1':3, '#n':64, '#act':'#act'}}],
        ['avg', {'stride':4}]])

plain_flower = CnnExtModel('plain_flower', fd,
        ['custom', {'name':'plain_flower', 'args':{'#act':'LAB'}}],
                    dump_structure=True)
plain_flower.exec_all(epoch_count=10, report=2)

CnnExtModel.set_macro('residual_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'stride':2}],
        ['custom', {'name':'rfull', 'args':{'#cnt':2, '#n':16, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':1, '#n':32, '#act':'#act'}}],
        ['custom', {'name':'rhalf', 'args':{'#cnt1':1, '#n':64, '#act':'#act'}}],
        ['avg', {'stride':4}]])

residual_flower = CnnExtModel('residual_flower', fd,
      ['custom', {'name':'residual_flower', 'args':{'#act':'LAB'}}],
                  dump_structure=True)
residual_flower.exec_all(epoch_count=10, report=2)

CnnExtModel.set_macro('bottleneck_flower',
    ['serial',
        ['conv', {'ksize':7, 'stride':2, 'chn':16, 'actions':'#act'}],
        ['max', {'ksize':3, 'stride':2}],
        ['custom', {'name':'bfull', 'args':{'#cnt':1,'#n1':16,'#n4': 64,
            '#act':'#act'}}],
        ['custom', {'name':'bhalf', 'args':{'#cnt1':2,'#n1':32,'#n4':128
            '#act':'#act'}}],
        ['custom', {'name':'bhalf', 'args':{'#cnt1':1,'#n1':64,'#n4':256,
            '#act':'#act'}}],
        ['avg', {'stride':4}]])

bottleneck_flower = CnnExtModel('bottleneck_flower', fd,
    ['custom', {'name':'bottleneck_flower', 'args':{'#act':'LAB'}}],
        dump_structure=True)
bottleneck_flower.exec_all(epoch_count=10, report=2)